In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import math

In [ ]:
def openfile(fileName):
    """
    Dado el nombre de un fichero devuelve la representación y las etiquetas
    """
    with open(fileName,"rb") as fh:
        representacion,etiquetas = pickle.load(fh)
    return representacion,etiquetas

In [ ]:
#Cargamos las mejores representaciones de cada
ncharTrain,charetiquetasTrain = openfile("ncharRepTrain")
ncharTest,charetiquetasTest = openfile("ncharRepTest")
print("n-gramas CHAR BOW --- OK")
w2vTrain,w2vetiquetasTrain = openfile("w2vRepTrain")
w2vTest,w2vetiquetasTest = openfile("w2vRepTest")
print("W2V REPR ------ OK")
pcarasTrain,pcarasetiquetasTrain = openfile("pcarasTrain")
pcarasTest,pcarasetiquetasTest = openfile("pcarasTest")
pcarasetiquetasTrain = [-1 if g == "male" else 1  for g in pcarasetiquetasTrain]
pcarasetiquetasTest = [-1 if g == "male" else 1  for g in pcarasetiquetasTest]
print("Porcentaje de caras de hombres en fotos --- OK")
bovwTrain,bovwetiquetasTrain = openfile("bovwRepTrain")
bovwTest,bovwetiquetasTest = openfile("bovwRepTest")
bovwetiquetasTrain = [-1 if g == "male" else 1  for g in bovwetiquetasTrain]
bovwetiquetasTest = [-1 if g == "male" else 1  for g in bovwetiquetasTest]
print("Bag of Visual Words --- OK")

In [ ]:
#Validamos que las etiquetas esten correctas
train = all(a == b == c ==d for a,b,c,d in zip(charetiquetasTrain,w2vetiquetasTrain,pcarasetiquetasTrain,bovwetiquetasTrain))
test = all(a == b == c == d for a,b,c,d in zip(charetiquetasTest,w2vetiquetasTest,pcarasetiquetasTest,bovwetiquetasTest))
print("Train: %r Test: %r " %(train,test))

# Esquema entrenamiento stacking:
## 1- Se entrenan los modelos
## 2- Se obtiene el nuevo conjunto de entrenamiento a partir de las predicciones
## 3- Se entrena el metaclasificador

# Fase 1:

In [ ]:
#Entreno modelo Char-ngram
nchar_clf = LogisticRegression(C=10)
nchar_clf.fit(ncharTrain,charetiquetasTrain)
print("OK! %.3f" % nchar_clf.score(ncharTest,charetiquetasTest))

In [ ]:
#Entreno modelo Word2Vec
w2v_clf = svm.SVC(C = 1000,kernel="linear")
w2v_clf.fit(w2vTrain,w2vetiquetasTrain)
print("OK! %.3f" %w2v_clf.score(w2vTest,w2vetiquetasTest))

In [ ]:
#Entreno modelo % hombres en fotos
pcaras_clf = svm.SVC(C = 1,kernel="rbf")
pcaras_clf.fit(np.array(pcarasTrain).reshape((3000,1)),pcarasetiquetasTrain)
print("OK! %.3f" %pcaras_clf.score(np.array(pcarasTest).reshape((2200,1)),pcarasetiquetasTest))

In [ ]:
#Entreno modelo basado en BOVW con VGG y reducida a 20 dimensiones con PCA
bovw_clf = LogisticRegression(C=1)
bovw_clf.fit(bovwTrain,bovwetiquetasTrain)
print("OK! %.3f" % bovw_clf.score(bovwTest,bovwetiquetasTest))

# Fase 2:

In [ ]:
#Construyo una matriz con las predicciones sobre el conjunto de train 
results = np.zeros((3000,4))
results[:,0] = nchar_clf.predict(ncharTrain)
results[:,1] = w2v_clf.predict(w2vTrain)
results[:,2] = pcaras_clf.predict(np.array(pcarasTrain).reshape((3000,1)))
results[:,3] = bovw_clf.predict(bovwTrain)

# Fase 3:

In [ ]:
#Entreno el metaclasificador 
from sklearn import tree
decision_tree = tree.DecisionTreeClassifier()
decision_tree.fit(results, charetiquetasTrain)
print("Meta-clasificador OK")

# Esquema test Stacking
## 1- Obtener las predicciones de  los clasificadores base almacenandolos en una matriz.
## 2- Obtener la clase más probable por el metaclasificador y evaluar precisión

# Fase 1:

In [ ]:
#Construyo una matriz con las predicciones sobre el conjunto de test 
resultsTest = np.zeros((2200,4))
resultsTest[:,0] = nchar_clf.predict(ncharTest)
resultsTest[:,1] = w2v_clf.predict(w2vTest)
resultsTest[:,2] = pcaras_clf.predict(np.array(pcarasTest).reshape((2200,1)))
resultsTest[:,3] = bovw_clf.predict(bovwTest)

# Fase 2:

In [ ]:
decision_tree.score(resultsTest,charetiquetasTest)

# Esquema training 2: Posible sobreajuste
## Adaptar el meta-modelo con otro conjunto de datos
## Conjunto de test ---> 20% adaptar metamodelo, 80% testing

In [ ]:
#Nuevos conjuntos de entrenamiento
etiquetasMETATest = charetiquetasTest[0:440]
etiquetasNEWTest = charetiquetasTest[440:-1]
ncharMETATest =  ncharTest[0:440]
ncharNEWTest = ncharTest[440:-1]
w2vMETATest = w2vTest[0:440]
w2vNEWTest = w2vTest[440:-1]
pcarasMETATest = np.array(pcarasTest).reshape((2200,1))[0:440]
pcarasNEWTest =  np.array(pcarasTest).reshape((2200,1))[440:-1]
bovwMETATest = bovwTest[0:440]
bovwNEWTest = bovwTest[440:-1]

## Fase2:

In [ ]:
#Construyo el nuevo conjunto de entrenamiento para el meta-modelo
results = np.zeros((440,4))
results[:,0] = nchar_clf.predict(ncharMETATest)
results[:,1] = w2v_clf.predict(w2vMETATest)
results[:,2] = pcaras_clf.predict(pcarasMETATest)
results[:,3] = bovw_clf.predict(bovwMETATest)

## Fase 3:

In [ ]:
from sklearn import tree
decision_tree = tree.DecisionTreeClassifier()
decision_tree.fit(results, etiquetasMETATest)
print("Meta-clasificador OK")

# Testing

In [ ]:
#Construyo una matriz con las predicciones sobre el conjunto de test 
resultsTest = np.zeros((1759,4))
resultsTest[:,0] = nchar_clf.predict(ncharNEWTest)
resultsTest[:,1] = w2v_clf.predict(w2vNEWTest)
resultsTest[:,2] = pcaras_clf.predict(pcarasNEWTest)
resultsTest[:,3] = bovw_clf.predict(bovwNEWTest)

In [ ]:
decision_tree.score(resultsTest,etiquetasNEWTest)